# 进程和线程

## Python中的进程 - multiprocessing

## Python中的线程 - threading

### 1. 首先看一个简单的多线程的例子:

In [1]:
import time
import threading

In [2]:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

In [3]:
print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


- 下面是一个多线程共同操作同一个变量的例子，定义了一个函数，该函数的功能是对一个全局变量进行累加，然后创建了两个线程，分别调用这个函数，最后打印出这个全局变量的值，可以看到，最后的结果是不确定的，因为两个线程都在对这个变量进行操作，所以最后的结果是不确定的。

In [4]:
# 假定这是你的银行存款:
balance = 0

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n

def run_thread(n):
    for i in range(2000000):
        change_it(n)

t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

18


- 我们可以通过给程序加入线程锁来解决这个问题。在一个线程操作的时候，其他线程必须等待，直到这个线程操作完毕。
- 锁的好处是确保了某段关键代码只能由一个线程从头到尾完整地执行，坏处当然也很多，首先是阻塞了其他线程，其次是由于可以存在多个锁，不同的线程持有不同的锁，并试图获取对方持有的锁时，可能会造成死锁，导致多个线程全部挂起，既不能执行，也无法结束，只能靠操作系统强制终止。

In [5]:
balance = 0
lock = threading.Lock()

def run_thread(n):
    for i in range(100000):
        # 先要获取锁: 只有一个线程可以成功获取锁，其他的线程就必须等待。
        lock.acquire()
        try:
            # 放心地改吧: 只有获得锁的线程才能改
            change_it(n)
        finally:
            # 改完了一定要释放锁: 否则其他的线程就会永远等待下去，成为了僵尸线程。
            lock.release()

t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


- 下面是一个简单的程序可以直观地展示锁的获取和释放过程。

In [8]:
LOCK = False
def run_thread_1():
   global LOCK
   while LOCK:
      print("run_thread_1 is waiting")
      time.sleep(1)
   LOCK = True   
   print("run_thread_1 lock the LOCK")
   print("run_thread_1 is doing something")
   time.sleep(5)
   LOCK = False   
   print("run_thread_1 release the LOCK")

def run_thread_2():
   global LOCK
   while LOCK:
      print("run_thread_2 is waiting")
      time.sleep(1)
   print("run_thread_2 lock the LOCK")
   LOCK = True   
   print("run_thread_2 is doing something")
   time.sleep(3)
   LOCK = False   
   print("run_thread_2 release the LOCK")
   
task1 = threading.Thread(target=run_thread_1)
task2 = threading.Thread(target=run_thread_2)
task1.start()
task2.start()
task1.join()
task2.join()
print("finish")

run_thread_1 lock the LOCK
run_thread_1 is doing something
run_thread_2 is waiting
run_thread_2 is waiting
run_thread_2 is waiting
run_thread_2 is waiting
run_thread_2 is waiting
run_thread_1 release the LOCK
run_thread_2 lock the LOCK
run_thread_2 is doing something
run_thread_2 release the LOCK
finish


- 接下来看一下多核CPU的情况下，多线程的效果，以及Python对多核CPU的支持。

In [9]:
import multiprocessing
multiprocessing.cpu_count()

8

In [11]:
def loop():
    x = 0
    while True:
        x = x ^ 1

for i in range(multiprocessing.cpu_count()):
    t = threading.Thread(target=loop)
    t.start()

- 可以发现，使用多线程的效果并不明显，这是因为Python的线程虽然是真正的线程，但解释器执行代码时，有一个GIL锁：Global Interpreter Lock，任何Python线程执行前，必须先获得GIL锁，然后，每执行100条字节码，解释器就自动释放GIL锁，让别的线程有机会执行。这个GIL全局锁实际上把所有线程的执行代码都给上了锁，所以，多线程在Python中只能交替执行，即使100个线程跑在100核CPU上，也只能用到1个核。

### 2. 一个简单的应用场景模拟

- 场景描述：在网上商城的结算阶段，服务器会首先检查支付情况，如何支付成功，就会给客户发送订单确认邮件，同时将浏览器的网页跳转到感谢页面。假设各个步骤花费的时间如下：
  - 检查支付：5秒
  - 发送邮件：10秒
  - 感谢页面：3秒
- 注意事项：三个步骤各自使用一个线程来处理，但是，如果没有确认支付成功，就不需要发送邮件和跳转到感谢页面。

In [7]:
import threading
import time

lock = threading.Lock()

class myThread1(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    
    def run(self):
        print("Starting " + self.name + "\n")
        lock.acquire()
        print_time(self.name, 1, self.counter)
        lock.release()
        print("Exiting " + self.name + "\n")

class myThread2(threading.Thread):
    def __init__(self, threadID, name, counter):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.counter = counter
    
    def run(self):
        print("Starting " + self.name + "\n")
        lock.acquire()
        lock.release()
        print_time(self.name, 1, self.counter)
        print("Exiting " + self.name + "\n")

def print_time(threadName, delay, counter):
    while counter:
        time.sleep(delay)
        print("%s: %s %s" % (threadName, time.ctime(time.time()), counter) + "\n")
        counter -= 1

# Create new threads
thread1 = myThread1(1, "checking payment", 5)
thread2 = myThread2(2, "sending email", 10)
thread3 = myThread2(3, "loading page", 3)

# Start new Threads
thread1.start()
thread2.start()
thread3.start()
thread1.join()
thread2.join()
thread3.join()
print("Exiting Main Thread")

Starting checking payment

Starting sending email

Starting loading page

checking payment: Mon Sep 26 21:33:30 2022 5

checking payment: Mon Sep 26 21:33:31 2022 4

checking payment: Mon Sep 26 21:33:32 2022 3

checking payment: Mon Sep 26 21:33:33 2022 2

checking payment: Mon Sep 26 21:33:34 2022 1

Exiting checking payment

loading page: Mon Sep 26 21:33:35 2022 3
sending email: Mon Sep 26 21:33:35 2022 10


sending email: Mon Sep 26 21:33:35 2022 9

loading page: Mon Sep 26 21:33:35 2022 2

loading page: Mon Sep 26 21:33:36 2022 1
sending email: Mon Sep 26 21:33:36 2022 8

Exiting loading page


sending email: Mon Sep 26 21:33:36 2022 7

sending email: Mon Sep 26 21:33:37 2022 6

sending email: Mon Sep 26 21:33:37 2022 5

sending email: Mon Sep 26 21:33:38 2022 4

sending email: Mon Sep 26 21:33:38 2022 3

sending email: Mon Sep 26 21:33:39 2022 2

sending email: Mon Sep 26 21:33:39 2022 1

Exiting sending email

Exiting Main Thread


In [1]:
from concurrent.futures import ThreadPoolExecutor
import time

def return_future(msg):
    time.sleep(3)
    return msg

pool = ThreadPoolExecutor(max_workers=2)
future = pool.submit(return_future, ("hello"))
print(future.done())
time.sleep(3)
print(future.done())
print(future.result())


False
False
hello


In [8]:
import concurrent.futures
import math

PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True


with ThreadPoolExecutor(max_workers=4) as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
        print('%d is prime: %s' % (number, prime))

# for number, prime in zip(PRIMES, map(is_prime, PRIMES)):
#     print('%d is prime: %s' % (number, prime))

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
